# 1º passo Gerar os dataset's 

• number of instances (rows in your dataset table) 

• number of classes (for multi-class classification problems) 

• proportion of classes (balanced and imbalanced class problem) 

• distribution of points within each class (shape of point clouds) 

• shape of the border between the class regions, from linear to whatever 

• level of noise

• level of overlap between the classes


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_blobs, make_moons, make_circles

